## Quick start: Custom Handler Example

### Import package

In [ ]:
import dxfeed as dx
from datetime import datetime  # for timed subscription
from dateutil.relativedelta import relativedelta

### Configure and create connection with Endpoint class
Create instance of Endpoint class which will connect provided address. 

In [ ]:
endpoint = dx.Endpoint('demo.dxfeed.com:7300')

### Configure and create subscription
You should specify event type. For timed subscription (conflated stream) you should also provide time to start subscription from.

In [ ]:
candle_sub = endpoint.create_subscription('Candle', date_time=datetime.now() - relativedelta(minutes=30))

In [ ]:
class DiffHandler(dx.EventHandler):
    def __init__(self):
        self.__prev_open = 0
        self.__prev_high = 0
        self.__prev_low = 0
        self.__prev_close = 0
        self.__prev_volume = 0
        self.volume_changes = list()
        self.counter = 0
        
    def update(self, event):
        if event[12] == event[12]:  # AskVolume not nan
            print(f'Symbol: {event[0]}')
            print(f'Open changed by: {event[5] - self.__prev_open}')
            self.__prev_open = event[5]
            print(f'High changed by: {event[6] - self.__prev_high}')
            self.__prev_high = event[6]
            print(f'Open changed by: {event[7] - self.__prev_low}')
            self.__prev_low = event[7]
            print(f'Close changed by: {event[8] - self.__prev_close}')
            self.__prev_close = event[8]
            # Volume logic
            vol_change = event[12] - self.__prev_volume
            self.volume_changes.append(vol_change)
            self.counter +=1
            print(f'Volume changed by: {vol_change}, from {self.__prev_volume}, to {event[12]}')
            self.__prev_volume = event[12]
            print(f'Ask events prcessed: {self.counter}')
            print('-------------------')
            if self.counter % 10 == 0:
                print(f'Average volume change for 10 past ask events is: {sum(self.volume_changes) / len(self.volume_changes)}')
                self.volume_changes.clear()
                print('-------------------')

In [ ]:
handler = DiffHandler()
candle_sub.set_event_handler(handler).add_symbols(['AAPL'])

### Close subscription

In [ ]:
candle_sub.close_subscription()

### Close connection

In [ ]:
endpoint.close_connection()
print(f'Connection status: {endpoint.connection_status}')